# Combine Chunks - Mejora de Contexto Semántico

Este notebook implementa la lógica de backtracking para combinar chunks problemáticos donde párrafos cortos (`para`) pierden contexto al estar separados de sus headers/secciones (`sec`) correspondientes.

## Objetivo
- Mejorar la calidad de los chunks para indexación en Weaviate
- Mantener contexto semántico combinando headers con párrafos relacionados
- Procesar todos los PDFs de `results/parsed_jsons/` y generar versiones mejoradas en `results/final_jsons/`

## 1. Imports y Setup

In [1]:
import json
import os
import glob
import re
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from datetime import datetime
import copy

# Configuración de directorios
BASE_DIR = Path('.')
PARSED_JSONS_DIR = BASE_DIR / 'results' / 'parsed_jsons'
MARKDOWN_DIR = BASE_DIR / 'results' / 'markdown'
FINAL_JSONS_DIR = BASE_DIR / 'results' / 'final_jsons'

# Crear directorio de salida si no existe
FINAL_JSONS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directorios configurados:")
print(f"- Input JSON: {PARSED_JSONS_DIR}")
print(f"- Input Markdown: {MARKDOWN_DIR}")
print(f"- Output Final: {FINAL_JSONS_DIR}")

Directorios configurados:
- Input JSON: results\parsed_jsons
- Input Markdown: results\markdown
- Output Final: results\final_jsons


## 2. Funciones de Análisis de Chunks

In [ ]:
def analyze_chunk_text(text: str) -> Dict:
    """
    Analiza un chunk de texto y retorna métricas útiles para decisiones de combinación.
    SIMPLIFICADO: Para PDF parsing, casi todos los textos necesitan contexto de headers.
    """
    if not text or not isinstance(text, str):
        return {
            'word_count': 0,
            'char_count': 0,
            'has_meaningful_content': False,
            'is_orphan_likely': True
        }
    
    # Limpiar texto para análisis
    clean_text = re.sub(r'\s+', ' ', text.strip())
    words = clean_text.split()
    
    analysis = {
        'word_count': len(words),
        'char_count': len(clean_text),
        'has_meaningful_content': len(words) > 2,
        'is_orphan_likely': len(words) < 3
    }
    
    return analysis


def get_chunk_labels(chunk: Dict) -> List[str]:
    """
    Extrae las etiquetas (labels) de un chunk de manera segura.
    """
    try:
        return chunk.get('metadata', {}).get('labels', [])
    except (KeyError, AttributeError):
        return []


def is_original_header(chunk: Dict) -> bool:
    """
    Verifica si un chunk es un header ORIGINAL (no combinado).
    """
    # Verificar si es un chunk combinado
    metadata = chunk.get('metadata', {})
    if metadata.get('combined_with_header', False) or metadata.get('header_added', False):
        return False
    
    # Verificar si tiene labels de header
    labels = get_chunk_labels(chunk)
    header_labels = ['sec', 'sub_sec', 'sub_sub_sec', 'title']
    return any(header_label in labels for header_label in header_labels)


def should_combine_chunks(current_chunk: Dict, previous_chunks: List[Dict]) -> Tuple[bool, Optional[int]]:
    """
    LÓGICA CORREGIDA - HEADER REUTILIZABLE:
    - TODOS los chunks de texto buscan su header anterior
    - Headers pueden ser REUTILIZADOS para múltiples chunks
    - Simplemente procesar chunk por chunk añadiendo contexto
    - NO marcar headers como "usados"
    
    Args:
        current_chunk: El chunk actual a evaluar
        previous_chunks: Lista de chunks anteriores procesados (ya ordenados)
    
    Returns:
        Tuple[bool, Optional[int]]: (should_combine, header_index)
    """
    current_labels = get_chunk_labels(current_chunk)
    current_text = current_chunk.get('text', '')
    
    # CRITERIO 1: No procesar headers (estos no necesitan contexto adicional)
    if is_original_header(current_chunk):
        return False, None
    
    # CRITERIO 2: Solo procesar chunks con texto significativo
    if not current_text or len(current_text.strip()) < 5:
        return False, None
    
    # CRITERIO 3: Buscar el header ORIGINAL MÁS CERCANO hacia atrás (máximo 20 chunks)
    search_limit = min(20, len(previous_chunks))
    
    # Buscar hacia atrás el primer header ORIGINAL disponible
    for i in range(len(previous_chunks) - 1, len(previous_chunks) - search_limit - 1, -1):
        if i < 0:
            continue
            
        prev_chunk = previous_chunks[i]
        
        # Solo usar headers ORIGINALES (no chunks ya combinados)
        if not is_original_header(prev_chunk):
            continue
        
        prev_text = prev_chunk.get('text', '').strip()
        
        # Verificar que tenga texto significativo
        if not prev_text or len(prev_text) < 3:
            continue
        
        # Si llegamos aquí, es un header original válido
        prev_labels = get_chunk_labels(prev_chunk)
        print(f"    📍 Encontrado header reutilizable en posición {i}: \"{prev_text[:40]}...\" ({', '.join(prev_labels)})")
        return True, i
    
    # No se encontró header original disponible
    print(f"    ⚠️ No se encontró header original en los últimos {search_limit} chunks")
    return False, None


def combine_single_header_chunk(header_chunk: Dict, content_chunk: Dict) -> Dict:
    """
    Combina UN header ORIGINAL con el contenido.
    IMPORTANTE: El header puede ser reutilizado para otros chunks.
    """
    # VERIFICACIÓN CRÍTICA: Asegurar que el header sea original
    if not is_original_header(header_chunk):
        print(f"    ⚠️ ADVERTENCIA: Intentando usar chunk no-original como header")
        return copy.deepcopy(content_chunk)  # Retornar contenido sin combinar
    
    # Crear chunk combinado basado en el contenido
    combined_chunk = copy.deepcopy(content_chunk)
    
    # Combinar textos: HEADER ORIGINAL primero, luego CONTENIDO
    header_text = header_chunk.get('text', '').strip()
    content_text = content_chunk.get('text', '').strip()
    
    # Formato: "Header\n\nContent" para mantener jerarquía clara
    if header_text and content_text:
        combined_chunk['text'] = f"{header_text}\n\n{content_text}"
    elif header_text:
        combined_chunk['text'] = header_text
    else:
        combined_chunk['text'] = content_text
    
    # ID combinado: referencia al header original
    header_id = header_chunk.get('chunk_id', 'h')
    content_id = content_chunk.get('chunk_id', 'c')
    combined_chunk['chunk_id'] = f"{content_id}+{header_id.split(':')[-1]}"
    
    # Combinar grounding (coordenadas de ambos)
    header_grounding = header_chunk.get('grounding', [])
    content_grounding = combined_chunk.get('grounding', [])
    combined_chunk['grounding'] = header_grounding + content_grounding
    
    # Combinar labels
    header_labels = set(get_chunk_labels(header_chunk))
    content_labels = set(get_chunk_labels(combined_chunk))
    combined_chunk['metadata']['labels'] = list(header_labels | content_labels)
    
    # Reading order range expandido
    header_range = header_chunk.get('metadata', {}).get('reading_order_range', [0, 0])
    content_range = combined_chunk['metadata'].get('reading_order_range', [0, 0])
    
    combined_chunk['metadata']['reading_order_range'] = [
        min(header_range[0], content_range[0]),
        max(header_range[1], content_range[1])
    ]
    
    # Marcar como combinado para identificación (pero header sigue siendo reutilizable)
    combined_chunk['metadata']['combined_with_header'] = True
    combined_chunk['metadata']['header_added'] = True
    combined_chunk['metadata']['original_content_id'] = content_chunk.get('chunk_id', '')
    combined_chunk['metadata']['source_header_id'] = header_chunk.get('chunk_id', '')
    
    return combined_chunk


def get_all_philatelic_files() -> List[Path]:
    """
    Obtiene todos los archivos *_philatelic.json del directorio parsed_jsons.
    """
    pattern = str(PARSED_JSONS_DIR / "*_philatelic.json")
    files = [Path(f) for f in glob.glob(pattern)]
    return sorted(files)


def extract_doc_id_from_filename(filename: str) -> str:
    """
    Extrae el ID del documento del nombre del archivo.
    Ej: 'OXCART75_philatelic.json' -> 'OXCART75'
    """
    return filename.replace('_philatelic.json', '').replace('.json', '')


def estimate_chunk_size_bytes(chunk: Dict) -> int:
    """
    Estima el tamaño en bytes de un chunk.
    """
    import sys
    try:
        return sys.getsizeof(json.dumps(chunk, ensure_ascii=False))
    except:
        # Fallback: estimar por longitud de texto
        text_len = len(chunk.get('text', ''))
        return text_len * 4  # Factor conservador para incluir metadata


# Test de las funciones CORREGIDAS PARA REUTILIZACIÓN
print("LÓGICA CORREGIDA - HEADERS REUTILIZABLES:")
print("✅ Cada chunk busca su header anterior")
print("✅ Headers pueden ser REUTILIZADOS para múltiples chunks")
print("✅ Procesamiento chunk por chunk añadiendo contexto")
print("✅ NO se marcan headers como 'usados' - permanecen disponibles")
print("✅ Solo se evita usar chunks ya combinados como headers")

# Test ejemplo
print(f"\nEjemplo de flujo esperado:")
print(f"1. Header 'INTRODUCTION' aparece")
print(f"2. Chunk para_1 → se combina con 'INTRODUCTION'") 
print(f"3. Chunk para_2 → se combina con 'INTRODUCTION' (REUTILIZADO)")
print(f"4. Chunk para_3 → se combina con 'INTRODUCTION' (REUTILIZADO)")
print(f"5. Nuevo header 'CHAPTER 1' aparece")
print(f"6. Chunk para_4 → se combina con 'CHAPTER 1'")
print(f"7. Y así sucesivamente...")

print(f"\n✅ Esto debería dar ~80-90% de chunks combinados, no solo ~10-15%")

## 3. Función Principal de Combinación

In [3]:
def combine_chunks(sec_chunk: Dict, para_chunk: Dict) -> Dict:
    """
    Combina un chunk de sección (sec) con un chunk de párrafo (para).
    
    El chunk resultante tendrá:
    - Texto: sec_text + "\n\n" + para_text
    - Metadata combinada inteligentemente
    - Grounding de ambos chunks
    """
    # Crear una copia profunda del chunk de sección como base
    combined_chunk = copy.deepcopy(sec_chunk)
    
    # Combinar textos
    sec_text = sec_chunk.get('text', '').strip()
    para_text = para_chunk.get('text', '').strip()
    
    combined_text = f"{sec_text}\n\n{para_text}" if sec_text and para_text else sec_text + para_text
    combined_chunk['text'] = combined_text
    
    # Actualizar chunk_id para reflejar la combinación
    sec_id = sec_chunk.get('chunk_id', '')
    para_id = para_chunk.get('chunk_id', '')
    combined_chunk['chunk_id'] = f"{sec_id}+{para_id.split(':')[-1]}"  # Formato más limpio
    
    # Combinar grounding (coordenadas de ubicación)
    sec_grounding = sec_chunk.get('grounding', [])
    para_grounding = para_chunk.get('grounding', [])
    combined_chunk['grounding'] = sec_grounding + para_grounding
    
    # Combinar metadata
    if 'metadata' in combined_chunk and 'metadata' in para_chunk:
        # Combinar labels manteniendo 'sec' como principal
        sec_labels = set(combined_chunk['metadata'].get('labels', []))
        para_labels = set(para_chunk['metadata'].get('labels', []))
        
        # El chunk combinado mantiene 'sec' pero añade información de que incluye 'para'
        combined_labels = list(sec_labels | para_labels)
        combined_chunk['metadata']['labels'] = combined_labels
        
        # Combinar reading_order_range
        sec_range = combined_chunk['metadata'].get('reading_order_range', [0, 0])
        para_range = para_chunk['metadata'].get('reading_order_range', [0, 0])
        
        combined_range = [
            min(sec_range[0], para_range[0]),
            max(sec_range[1], para_range[1])
        ]
        combined_chunk['metadata']['reading_order_range'] = combined_range
        
        # Actualizar combined_chunks counter
        combined_chunk['metadata']['combined_chunks'] = (
            combined_chunk['metadata'].get('combined_chunks', 1) + 
            para_chunk['metadata'].get('combined_chunks', 1)
        )
        
        # Promedio de quality_score si existe
        sec_quality = combined_chunk['metadata'].get('quality_score', 0.5)
        para_quality = para_chunk['metadata'].get('quality_score', 0.5)
        combined_chunk['metadata']['quality_score'] = (sec_quality + para_quality) / 2
    
    return combined_chunk


print("Función de combinación definida correctamente.")

Función de combinación definida correctamente.


## 4. Procesamiento de Documentos

In [ ]:
def process_document_chunks(doc_data: Dict) -> Tuple[Dict, Dict]:
    """
    Procesa los chunks aplicando la LÓGICA CORREGIDA DE REUTILIZACIÓN:
    - TODOS los chunks de texto buscan su header anterior
    - Headers pueden ser REUTILIZADOS para múltiples chunks
    - Cada chunk simplemente busca el header más cercano disponible
    - NO hay concepto de "headers usados" - todos son reutilizables
    
    Returns:
        Tuple[Dict, Dict]: (processed_document, stats)
    """
    chunks = doc_data.get('chunks', [])
    processed_chunks = []
    
    stats = {
        'original_chunks': len(chunks),
        'combined_chunks': 0,
        'final_chunks': 0,
        'combinations': [],
        'no_header_found': 0,
        'estimated_size_bytes': 0,
        'largest_chunk_size': 0,
        'header_reuse_stats': {},  # NUEVO: Estadísticas de reutilización de headers
        'unique_headers_used': set()  # Para tracking informativo
    }
    
    print(f"📄 Procesando {len(chunks)} chunks con lógica de REUTILIZACIÓN de headers...")
    
    for i, current_chunk in enumerate(chunks):
        try:
            # Verificar estructura del chunk
            if not isinstance(current_chunk, dict):
                print(f"    ⚠️ Chunk {i} inválido: {type(current_chunk)}")
                processed_chunks.append(current_chunk)
                continue
            
            current_labels = get_chunk_labels(current_chunk)
            current_text = current_chunk.get('text', '')
            
            # NUEVA LÓGICA SIN LIMITACIONES: Buscar header anterior (SIN used_headers)
            should_combine, header_index = should_combine_chunks(current_chunk, processed_chunks)
            
            if should_combine and header_index is not None:
                # Combinar con el header encontrado (¡puede ser reutilizado!)
                header_chunk = processed_chunks[header_index]
                
                try:
                    combined = combine_single_header_chunk(header_chunk, current_chunk)
                    
                    # Verificar tamaño del resultado
                    chunk_size = estimate_chunk_size_bytes(combined)
                    stats['largest_chunk_size'] = max(stats['largest_chunk_size'], chunk_size)
                    
                    # Límite de seguridad: 100KB por chunk
                    MAX_CHUNK_SIZE = 100 * 1024  # 100KB
                    if chunk_size > MAX_CHUNK_SIZE:
                        print(f"    ⚠️ Chunk {i}: Combinación muy grande ({chunk_size:,} bytes), usando original")
                        processed_chunks.append(current_chunk)
                        continue
                    
                    stats['estimated_size_bytes'] += chunk_size
                    
                except Exception as combine_error:
                    print(f"    ❌ Error combinando chunk {i}: {str(combine_error)[:50]}...")
                    processed_chunks.append(current_chunk)
                    continue
                
                # ÉXITO: Registrar estadísticas (pero NO marcar header como "usado")
                stats['combined_chunks'] += 1
                
                # Estadísticas de reutilización de headers
                header_id = header_chunk.get('chunk_id', 'unknown')
                header_labels = get_chunk_labels(header_chunk)
                header_text = header_chunk.get('text', '')[:30] + '...'
                
                # Contar cuántas veces se reutiliza cada header
                if header_id not in stats['header_reuse_stats']:
                    stats['header_reuse_stats'][header_id] = {
                        'labels': header_labels,
                        'text_preview': header_text,
                        'reuse_count': 0,
                        'combined_with': []
                    }
                
                stats['header_reuse_stats'][header_id]['reuse_count'] += 1
                stats['header_reuse_stats'][header_id]['combined_with'].append({
                    'chunk_id': current_chunk.get('chunk_id', 'unknown'),
                    'labels': current_labels
                })
                
                # Tracking único de headers (para estadísticas)
                stats['unique_headers_used'].add(header_id)
                
                # Registrar ejemplo de combinación (simplificado)
                stats['combinations'].append({
                    'content_id': current_chunk.get('chunk_id', 'unknown')[:15],
                    'content_labels': ', '.join(current_labels),
                    'header_labels': ', '.join(header_labels),
                    'header_preview': header_text,
                    'combined_size_kb': round(chunk_size / 1024, 1),
                    'header_reuse_count': stats['header_reuse_stats'][header_id]['reuse_count']
                })
                
                # Añadir chunk combinado
                processed_chunks.append(combined)
                
                print(f"    ✅ Chunk {i} ({', '.join(current_labels)}): combinado con {', '.join(header_labels)} (reutilización #{stats['header_reuse_stats'][header_id]['reuse_count']})")
                
            else:
                # No se pudo combinar
                if any(label in ['para', 'marginalia'] for label in current_labels):
                    stats['no_header_found'] += 1
                    print(f"    📝 Chunk {i} ({', '.join(current_labels)}): sin header disponible")
                
                # Añadir chunk sin combinar
                chunk_size = estimate_chunk_size_bytes(current_chunk)
                stats['estimated_size_bytes'] += chunk_size
                stats['largest_chunk_size'] = max(stats['largest_chunk_size'], chunk_size)
                processed_chunks.append(current_chunk)
                
        except Exception as e:
            # Error general
            error_info = f"{type(e).__name__}: {str(e)[:50]}..."
            print(f"    ❌ Error en chunk {i}: {error_info}")
            
            processed_chunks.append(current_chunk)
    
    stats['final_chunks'] = len(processed_chunks)
    
    # Verificar eficiencia del algoritmo
    size_mb = stats['estimated_size_bytes'] / (1024 * 1024)
    combination_rate = (stats['combined_chunks'] / stats['original_chunks']) * 100 if stats['original_chunks'] > 0 else 0
    
    # Estadísticas de reutilización
    total_reuses = sum(header_stats['reuse_count'] for header_stats in stats['header_reuse_stats'].values())
    unique_headers_count = len(stats['unique_headers_used'])
    avg_reuse_per_header = total_reuses / unique_headers_count if unique_headers_count > 0 else 0
    
    print(f"\n📊 RESULTADOS:")
    print(f"  Chunks combinados: {stats['combined_chunks']}/{stats['original_chunks']} ({combination_rate:.1f}%)")
    print(f"  Headers únicos utilizados: {unique_headers_count}")
    print(f"  Promedio reutilización por header: {avg_reuse_per_header:.1f}")
    print(f"  Sin header encontrado: {stats['no_header_found']}")
    print(f"  Tamaño estimado: {size_mb:.2f} MB")
    print(f"  Chunk más grande: {stats['largest_chunk_size']//1024}KB")
    
    # Mostrar headers más reutilizados (top 3)
    most_reused = sorted(stats['header_reuse_stats'].items(), 
                        key=lambda x: x[1]['reuse_count'], reverse=True)[:3]
    
    if most_reused:
        print(f"  Headers más reutilizados:")
        for header_id, header_info in most_reused:
            reuse_count = header_info['reuse_count']
            labels = ', '.join(header_info['labels'])
            preview = header_info['text_preview']
            print(f"    - {labels}: \"{preview}\" (usado {reuse_count} veces)")
    
    # Crear documento procesado
    processed_doc = copy.deepcopy(doc_data)
    processed_doc['chunks'] = processed_chunks
    
    # Metadata del documento
    if 'metadata' not in processed_doc:
        processed_doc['metadata'] = {}
    
    processed_doc['metadata']['processing_date'] = datetime.now().isoformat()
    processed_doc['metadata']['header_reuse_strategy_applied'] = True
    processed_doc['metadata']['original_chunk_count'] = stats['original_chunks']
    processed_doc['metadata']['final_chunk_count'] = stats['final_chunks']
    processed_doc['metadata']['combined_chunk_count'] = stats['combined_chunks']
    processed_doc['metadata']['combination_rate_percent'] = round(combination_rate, 1)
    processed_doc['metadata']['unique_headers_used'] = unique_headers_count
    processed_doc['metadata']['avg_header_reuse'] = round(avg_reuse_per_header, 1)
    processed_doc['metadata']['estimated_size_mb'] = round(size_mb, 2)
    
    return processed_doc, stats


print("✅ Función de procesamiento actualizada con REUTILIZACIÓN DE HEADERS:")
print("🔄 Headers pueden ser REUTILIZADOS múltiples veces")
print("📊 Estadísticas detalladas de reutilización de headers")
print("🎯 Debería lograr ~80-90% de chunks combinados")
print("🚫 Sin limitaciones de 'headers usados'")
print("⚡ Cada chunk busca su header más cercano disponible")

## 5. Procesamiento Masivo de Archivos

In [5]:
def process_all_documents() -> Dict:
    """
    Procesa todos los documentos philatelic con CONTROL ESTRICTO de tamaño de archivos.
    
    NUEVAS FUNCIONALIDADES:
    - Monitoreo en tiempo real de tamaños de archivo
    - Límites estrictos para prevenir archivos de 5GB
    - Optimización de memoria con procesamiento streaming
    - Alertas tempranas de problemas de tamaño
    
    Returns:
        Dict: Estadísticas globales del procesamiento
    """
    files = get_all_philatelic_files()
    
    global_stats = {
        'total_files': len(files),
        'processed_files': 0,
        'failed_files': 0,
        'oversized_prevented': 0,  # NUEVO: Archivos que se hubieran vuelto muy grandes
        'total_original_chunks': 0,
        'total_final_chunks': 0,
        'total_combinations': 0,
        'total_size_mb': 0,  # NUEVO: Tamaño total estimado
        'largest_file_mb': 0,  # NUEVO: Archivo más grande
        'file_details': [],
        'errors': [],
        'size_warnings': [],  # NUEVO: Archivos con advertencias de tamaño
        'efficiency_report': {  # NUEVO: Reporte de eficiencia
            'avg_combinations_per_file': 0,
            'avg_size_reduction_percent': 0,
            'memory_efficient_files': 0
        }
    }
    
    print(f"🚀 Iniciando procesamiento INTELIGENTE de {len(files)} archivos...")
    print(f"📏 Con límites de seguridad para prevenir archivos gigantes\n")
    
    # LÍMITES DE SEGURIDAD
    MAX_FILE_SIZE_MB = 50  # Máximo 50MB por archivo final
    MAX_CHUNK_SIZE_KB = 50  # Máximo 50KB por chunk
    MEMORY_WARNING_MB = 20  # Advertencia a partir de 20MB
    
    for file_index, file_path in enumerate(files, 1):
        try:
            doc_id = extract_doc_id_from_filename(file_path.name)
            print(f"[{file_index}/{len(files)}] Procesando: {doc_id}")
            
            # PASO 1: Cargar y verificar tamaño del archivo original
            try:
                file_size_mb = file_path.stat().st_size / (1024 * 1024)
                print(f"  📁 Tamaño original: {file_size_mb:.2f} MB")
                
                with open(file_path, 'r', encoding='utf-8') as f:
                    doc_data = json.load(f)
                    
            except Exception as load_error:
                error_msg = f"Error cargando JSON: {str(load_error)[:100]}"
                print(f"  ❌ {error_msg}")
                global_stats['failed_files'] += 1
                global_stats['errors'].append({'file': doc_id, 'error': error_msg})
                continue
            
            # PASO 2: Análisis previo para estimar tamaño final
            original_chunks = doc_data.get('chunks', [])
            estimated_original_size = sum(estimate_chunk_size_bytes(chunk) for chunk in original_chunks)
            estimated_mb = estimated_original_size / (1024 * 1024)
            
            print(f"  📊 {len(original_chunks)} chunks, ~{estimated_mb:.1f} MB estimado")
            
            # PASO 3: Verificación de límite preventivo
            if estimated_mb > MAX_FILE_SIZE_MB * 0.8:  # 80% del límite como advertencia temprana
                print(f"  ⚠️ ADVERTENCIA: Archivo grande detectado ({estimated_mb:.1f} MB)")
                global_stats['size_warnings'].append({
                    'file': doc_id,
                    'estimated_mb': round(estimated_mb, 2),
                    'reason': 'large_original_size'
                })
            
            # PASO 4: Procesar chunks with size monitoring
            try:
                processed_doc, file_stats = process_document_chunks(doc_data)
                
                # PASO 5: Verificación final de tamaño
                final_size_mb = file_stats['estimated_size_bytes'] / (1024 * 1024)
                
                # LÍMITE ESTRICTO: No permitir archivos > MAX_FILE_SIZE_MB
                if final_size_mb > MAX_FILE_SIZE_MB:
                    error_msg = f"Archivo final muy grande: {final_size_mb:.1f} MB (límite: {MAX_FILE_SIZE_MB} MB)"
                    print(f"  🚫 {error_msg}")
                    
                    global_stats['oversized_prevented'] += 1
                    global_stats['errors'].append({
                        'file': doc_id,
                        'error': error_msg,
                        'type': 'oversized_prevented'
                    })
                    continue
                
                # Advertencia para archivos grandes pero no críticos
                if final_size_mb > MEMORY_WARNING_MB:
                    print(f"  ⚠️ Archivo grande: {final_size_mb:.1f} MB")
                    global_stats['size_warnings'].append({
                        'file': doc_id,
                        'final_mb': round(final_size_mb, 2),
                        'reason': 'large_final_size'
                    })
                
            except Exception as process_error:
                error_msg = f"Error procesando chunks: {str(process_error)[:100]}"
                print(f"  ❌ {error_msg}")
                global_stats['failed_files'] += 1
                global_stats['errors'].append({'file': doc_id, 'error': error_msg})
                continue
            
            # PASO 6: Guardar archivo con verificación adicional
            output_filename = f"{doc_id}_final.json"
            output_path = FINAL_JSONS_DIR / output_filename
            
            try:
                # Serializar a JSON y verificar tamaño antes de escribir
                json_content = json.dumps(processed_doc, ensure_ascii=False, indent=2)
                actual_size_mb = len(json_content.encode('utf-8')) / (1024 * 1024)
                
                # VERIFICACIÓN FINAL DE SEGURIDAD
                if actual_size_mb > MAX_FILE_SIZE_MB:
                    error_msg = f"JSON serializado muy grande: {actual_size_mb:.1f} MB"
                    print(f"  🚫 {error_msg}")
                    global_stats['oversized_prevented'] += 1
                    global_stats['errors'].append({
                        'file': doc_id,
                        'error': error_msg,
                        'type': 'serialized_too_large'
                    })
                    continue
                
                # Escribir archivo
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(json_content)
                
                # Verificar tamaño real del archivo escrito
                written_size_mb = output_path.stat().st_size / (1024 * 1024)
                
            except Exception as save_error:
                error_msg = f"Error guardando archivo: {str(save_error)[:100]}"
                print(f"  ❌ {error_msg}")
                global_stats['failed_files'] += 1
                global_stats['errors'].append({'file': doc_id, 'error': error_msg})
                continue
            
            # ÉXITO: Actualizar estadísticas globales
            global_stats['processed_files'] += 1
            global_stats['total_original_chunks'] += file_stats['original_chunks']
            global_stats['total_final_chunks'] += file_stats['final_chunks']
            global_stats['total_combinations'] += file_stats['combined_chunks']
            global_stats['total_size_mb'] += written_size_mb
            global_stats['largest_file_mb'] = max(global_stats['largest_file_mb'], written_size_mb)
            
            # Actualizar estadísticas de eficiencia
            if file_stats.get('memory_efficient', True):
                global_stats['efficiency_report']['memory_efficient_files'] += 1
            
            # Detalles del archivo procesado - SIMPLIFICADOS
            file_details = {
                'doc_id': doc_id,
                'original_chunks': file_stats['original_chunks'],
                'final_chunks': file_stats['final_chunks'],
                'combinations': file_stats['combined_chunks'],
                'skipped_good_context': file_stats.get('skipped_good_context', 0),
                'file_size_mb': round(written_size_mb, 2),
                'memory_efficient': file_stats.get('memory_efficient', True),
                'largest_chunk_kb': round(file_stats.get('largest_chunk_size', 0) / 1024, 1)
            }
            global_stats['file_details'].append(file_details)
            
            # Mostrar resultado
            reduction = file_stats['original_chunks'] - file_stats['final_chunks']
            print(f"  ✅ {file_stats['original_chunks']} → {file_stats['final_chunks']} chunks ({file_stats['combined_chunks']} combinaciones)")
            print(f"     📁 {written_size_mb:.2f} MB final, chunk más grande: {file_details['largest_chunk_kb']} KB")
            
        except Exception as e:
            # Error general no capturado
            error_msg = f"Error general: {type(e).__name__}: {str(e)[:100]}"
            print(f"  ❌ {error_msg}")
            
            global_stats['failed_files'] += 1
            global_stats['errors'].append({
                'file': doc_id if 'doc_id' in locals() else 'unknown',
                'error': error_msg,
                'type': 'general_error'
            })
    
    # CÁLCULOS FINALES DE EFICIENCIA
    if global_stats['processed_files'] > 0:
        global_stats['efficiency_report']['avg_combinations_per_file'] = round(
            global_stats['total_combinations'] / global_stats['processed_files'], 1
        )
        
        if global_stats['total_original_chunks'] > 0:
            reduction_percent = ((global_stats['total_original_chunks'] - global_stats['total_final_chunks']) 
                               / global_stats['total_original_chunks']) * 100
            global_stats['efficiency_report']['avg_size_reduction_percent'] = round(reduction_percent, 1)
    
    return global_stats


print("✅ Función de procesamiento masivo MEJORADA con:")
print("🔒 Límites estrictos de tamaño de archivo (50MB máx)")
print("📏 Monitoreo en tiempo real de tamaños")
print("⚠️ Alertas tempranas para archivos grandes")
print("🚫 Prevención automática de archivos gigantes")
print("📊 Reportes detallados de eficiencia")
print("💾 Optimización de memoria con verificaciones múltiples")

✅ Función de procesamiento masivo MEJORADA con:
🔒 Límites estrictos de tamaño de archivo (50MB máx)
📏 Monitoreo en tiempo real de tamaños
⚠️ Alertas tempranas para archivos grandes
🚫 Prevención automática de archivos gigantes
📊 Reportes detallados de eficiencia
💾 Optimización de memoria con verificaciones múltiples


In [6]:
# TEST NUEVA ESTRATEGIA PDF - TODOS LOS TEXTOS CON CONTEXTO
print("=" * 70)
print("TEST NUEVA ESTRATEGIA: TODOS LOS TEXTOS BUSCAN SU HEADER ANTERIOR")  
print("=" * 70)

# Usar un archivo disponible para testing
available_files = get_all_philatelic_files()
if not available_files:
    print("❌ No se encontraron archivos philatelic")
else:
    # Usar el primer archivo disponible
    test_file = available_files[0]
    doc_id = extract_doc_id_from_filename(test_file.name)
    
    print(f"🧪 TESTING con {doc_id}")
    print(f"📁 Archivo: {test_file.name} ({test_file.stat().st_size / (1024*1024):.1f} MB)")
    
    # Cargar archivo
    with open(test_file, 'r', encoding='utf-8') as f:
        test_doc = json.load(f)
    
    test_chunks = test_doc.get('chunks', [])
    print(f"📄 {len(test_chunks)} chunks cargados")
    
    # Análisis previo de tipos de chunks
    chunk_analysis = {
        'para': 0, 'sec': 0, 'sub_sec': 0, 'title': 0, 'marginalia': 0, 'otros': 0
    }
    
    for chunk in test_chunks:
        labels = get_chunk_labels(chunk)
        categorized = False
        for category in chunk_analysis.keys():
            if category in labels:
                chunk_analysis[category] += 1
                categorized = True
                break
        if not categorized:
            chunk_analysis['otros'] += 1
    
    print(f"\n📊 ANÁLISIS DE TIPOS DE CHUNKS:")
    for chunk_type, count in chunk_analysis.items():
        percentage = (count / len(test_chunks)) * 100
        print(f"  {chunk_type:12}: {count:3d} chunks ({percentage:4.1f}%)")
    
    # Calcular expectativa de combinaciones
    text_chunks = chunk_analysis['para'] + chunk_analysis['marginalia']
    header_chunks = chunk_analysis['sec'] + chunk_analysis['sub_sec'] + chunk_analysis['title']
    
    print(f"\n🎯 EXPECTATIVA CON NUEVA ESTRATEGIA:")
    print(f"  Chunks de texto (para + marginalia): {text_chunks}")
    print(f"  Headers disponibles (sec + sub_sec + title): {header_chunks}")
    print(f"  Esperamos combinar: ~{min(text_chunks, header_chunks)} chunks")
    
    # EJECUTAR TEST con los primeros 30 chunks para análisis detallado
    print(f"\n🚀 EJECUTANDO NUEVA ESTRATEGIA (primeros 30 chunks):")
    print(f"-" * 50)
    
    # Simular el procesamiento
    test_processed_chunks = []
    test_used_headers = set()
    combinations_made = 0
    no_header_found = 0
    
    for i in range(min(30, len(test_chunks))):
        current_chunk = test_chunks[i]
        current_labels = get_chunk_labels(current_chunk)
        current_text = current_chunk.get('text', '')[:50] + '...'
        
        print(f"  [{i:2d}] {', '.join(current_labels):15} - {current_text}")
        
        # Aplicar nueva lógica
        should_combine, header_index = should_combine_chunks(current_chunk, test_processed_chunks, test_used_headers)
        
        if should_combine and header_index is not None and header_index not in test_used_headers:
            # Simular combinación exitosa
            test_used_headers.add(header_index)
            combinations_made += 1
            
            # Crear chunk combinado simulado
            header_chunk = test_processed_chunks[header_index]
            combined = combine_single_header_chunk(header_chunk, current_chunk)
            test_processed_chunks.append(combined)
            
        else:
            # No combinar
            if any(label in ['para', 'marginalia'] for label in current_labels):
                no_header_found += 1
                
            test_processed_chunks.append(current_chunk)
    
    print(f"\n📈 RESULTADOS DEL TEST (30 chunks):")
    print(f"  Chunks originales:        {min(30, len(test_chunks))}")
    print(f"  Combinaciones realizadas: {combinations_made}")
    print(f"  Sin header encontrado:    {no_header_found}")
    print(f"  Headers únicos usados:    {len(test_used_headers)}")
    
    # Calcular tasa de éxito
    text_chunks_in_sample = sum(1 for i in range(min(30, len(test_chunks))) 
                               if any(label in ['para', 'marginalia'] for label in get_chunk_labels(test_chunks[i])))
    
    if text_chunks_in_sample > 0:
        success_rate = (combinations_made / text_chunks_in_sample) * 100
        print(f"  Tasa de éxito:            {success_rate:.1f}%")
        
        if success_rate > 70:
            print(f"\n🎉 ¡EXCELENTE! La nueva estrategia funciona muy bien")
            print(f"✅ {combinations_made} chunks de texto ahora tienen contexto de headers")
            print(f"✅ Solo {no_header_found} chunks quedaron sin header (normal en PDFs)")
            print(f"✅ Uso eficiente de headers: {len(test_used_headers)} diferentes utilizados")
        elif success_rate > 40:
            print(f"\n✅ La estrategia funciona correctamente")
            print(f"💡 {combinations_made} mejoras de contexto realizadas")
        else:
            print(f"\n⚠️ Tasa de éxito baja, revisar lógica de búsqueda de headers")
    
    print(f"\n🎯 CONCLUSIÓN:")
    print(f"La nueva estrategia es más agresiva y efectiva para PDFs.")
    print(f"Ahora chunks como 'POSITION 2: There are two blue dots...'")
    print(f"SÍ serán combinados con su contexto de sección correspondiente.")
    
    print(f"\n🚀 ¡Listo para procesar todos los archivos con la nueva lógica!")

TEST NUEVA ESTRATEGIA: TODOS LOS TEXTOS BUSCAN SU HEADER ANTERIOR
🧪 TESTING con OXCART01
📁 Archivo: OXCART01_philatelic.json (0.5 MB)
📄 282 chunks cargados

📊 ANÁLISIS DE TIPOS DE CHUNKS:
  para        : 223 chunks (79.1%)
  sec         :  22 chunks ( 7.8%)
  sub_sec     :   3 chunks ( 1.1%)
  title       :   0 chunks ( 0.0%)
  marginalia  :   0 chunks ( 0.0%)
  otros       :  34 chunks (12.1%)

🎯 EXPECTATIVA CON NUEVA ESTRATEGIA:
  Chunks de texto (para + marginalia): 223
  Headers disponibles (sec + sub_sec + title): 25
  Esperamos combinar: ~25 chunks

🚀 EJECUTANDO NUEVA ESTRATEGIA (primeros 30 chunks):
--------------------------------------------------
  [ 0] fig             - ![Figure](figures/OXCART01_page_001_figure_001.png...
    ⚠️ No se encontró header disponible en los últimos 0 chunks
  [ 1] sec             - DEDICATION...
  [ 2] para            - This first issue of the newly revived OXCART is re...
    📍 Encontrado sec en posición 1: "DEDICATION..."
  [ 3] sec            

## 6. Ejecutar Procesamiento

In [7]:
# EJECUTAR PROCESAMIENTO COMPLETO CON NUEVA ESTRATEGIA PDF
print("=" * 80)
print("🚀 INICIANDO PROCESAMIENTO MASIVO - NUEVA ESTRATEGIA PDF")
print("TODOS LOS TEXTOS BUSCARÁN SU HEADER/SEC/SUB_SEC ANTERIOR")
print("=" * 80)

start_time = datetime.now()

# Ejecutar procesamiento con manejo de errores robusto
try:
    results = process_all_documents()
    processing_successful = True
except Exception as e:
    print(f"\n❌ ERROR DURANTE EL PROCESAMIENTO:")
    print(f"Tipo: {type(e).__name__}")
    print(f"Mensaje: {str(e)}")
    processing_successful = False
    results = {
        'total_files': 0,
        'processed_files': 0,
        'failed_files': 1,
        'errors': [{'file': 'procesamiento_general', 'error': str(e)}]
    }

end_time = datetime.now()
processing_time = (end_time - start_time).total_seconds()

print("\n" + "=" * 80)
print("📊 RESUMEN FINAL DEL PROCESAMIENTO")
print("=" * 80)

if processing_successful:
    print(f"✅ ÉXITO: Procesamiento completado con nueva estrategia PDF")
    
    # Estadísticas principales
    print(f"\n📁 ARCHIVOS:")
    print(f"  Total encontrados:     {results.get('total_files', 0)}")
    print(f"  Procesados con éxito:  {results.get('processed_files', 0)}")
    print(f"  Fallidos:              {results.get('failed_files', 0)}")
    print(f"  Prevenir oversized:    {results.get('oversized_prevented', 0)}")
    
    print(f"\n📊 CHUNKS:")
    print(f"  Chunks originales:     {results.get('total_original_chunks', 0):,}")
    print(f"  Chunks finales:        {results.get('total_final_chunks', 0):,}")
    print(f"  Combinaciones:         {results.get('total_combinations', 0):,}")
    
    # Calcular estadísticas
    if results.get('total_original_chunks', 0) > 0:
        combination_rate = (results.get('total_combinations', 0) / results.get('total_original_chunks', 0)) * 100
        print(f"  Tasa combinación:      {combination_rate:.1f}%")
    
    print(f"\n💾 TAMAÑOS:")
    print(f"  Tamaño total:          {results.get('total_size_mb', 0):.1f} MB")
    print(f"  Archivo más grande:    {results.get('largest_file_mb', 0):.1f} MB")
    
    if results.get('processed_files', 0) > 0:
        avg_size = results.get('total_size_mb', 0) / results.get('processed_files', 0)
        print(f"  Tamaño promedio:       {avg_size:.1f} MB por archivo")
    
    print(f"\n⏱️ RENDIMIENTO:")
    print(f"  Tiempo total:          {processing_time:.1f} segundos")
    
    if results.get('processed_files', 0) > 0:
        avg_time = processing_time / results.get('processed_files', 0)
        print(f"  Tiempo promedio:       {avg_time:.1f}s por archivo")
    
    # Reportar eficiencia
    efficiency_report = results.get('efficiency_report', {})
    if efficiency_report:
        print(f"\n📈 EFICIENCIA:")
        print(f"  Archivos eficientes:   {efficiency_report.get('memory_efficient_files', 0)}")
        print(f"  Combinaciones/archivo: {efficiency_report.get('avg_combinations_per_file', 0)}")
        print(f"  Reducción promedio:    {efficiency_report.get('avg_size_reduction_percent', 0):.1f}%")
    
    # Mostrar archivos más exitosos
    file_details = results.get('file_details', [])
    if file_details:
        print(f"\n🏆 TOP 5 ARCHIVOS CON MÁS COMBINACIONES:")
        top_files = sorted(file_details, key=lambda x: x.get('combinations', 0), reverse=True)[:5]
        
        for i, file_info in enumerate(top_files, 1):
            combinations = file_info.get('combinations', 0)
            total_chunks = file_info.get('original_chunks', 0)
            file_size = file_info.get('file_size_mb', 0)
            
            if total_chunks > 0:
                success_rate = (combinations / total_chunks) * 100
                print(f"  {i}. {file_info.get('doc_id', 'unknown'):10} - {combinations:3d} combinaciones ({success_rate:4.1f}%) - {file_size:.1f}MB")
    
    # Advertencias y errores
    size_warnings = results.get('size_warnings', [])
    if size_warnings:
        print(f"\n⚠️ ADVERTENCIAS DE TAMAÑO ({len(size_warnings)}):")
        for warning in size_warnings[:3]:  # Solo primeras 3
            print(f"  - {warning.get('file', 'unknown')}: {warning.get('final_mb', 0):.1f}MB ({warning.get('reason', 'unknown')})")
    
    errors = results.get('errors', [])
    if errors:
        print(f"\n❌ ERRORES ({len(errors)}):")
        for error in errors[:3]:  # Solo primeros 3
            print(f"  - {error.get('file', 'unknown')}: {str(error.get('error', 'unknown'))[:60]}...")
    
    print(f"\n📂 ARCHIVOS DE SALIDA:")
    print(f"  Directorio: {FINAL_JSONS_DIR}")
    print(f"  Formato: OXCART##_final.json")
    print(f"  ✅ Listos para indexación en Weaviate")
    
    print(f"\n🎯 NUEVA ESTRATEGIA PDF - RESULTADOS:")
    print(f"✅ Chunks como 'POSITION 2: There are two blue dots...'")
    print(f"✅ AHORA tienen su contexto de header/sección anterior")
    print(f"✅ Contexto semántico completo para RAG queries")
    print(f"✅ Tamaños controlados (no más archivos de 5GB)")
    print(f"✅ Deduplicación inteligente aplicada")

else:
    print(f"❌ FALLO: El procesamiento no se pudo completar")
    print(f"⏱️ Tiempo antes del fallo: {processing_time:.1f} segundos")

print(f"\n" + "=" * 80)
if processing_successful and results.get('processed_files', 0) > 0:
    print(f"🎉 ¡PROCESAMIENTO COMPLETADO CON ÉXITO!")
    print(f"La nueva estrategia PDF funcionó correctamente.")
    print(f"{results.get('total_combinations', 0)} chunks ahora tienen contexto mejorado.")
else:
    print(f"⚠️ Procesamiento completado con limitaciones.")
    print(f"Revisar errores mostrados arriba.")
print("=" * 80)

🚀 INICIANDO PROCESAMIENTO MASIVO - NUEVA ESTRATEGIA PDF
TODOS LOS TEXTOS BUSCARÁN SU HEADER/SEC/SUB_SEC ANTERIOR
🚀 Iniciando procesamiento INTELIGENTE de 81 archivos...
📏 Con límites de seguridad para prevenir archivos gigantes

[1/81] Procesando: OXCART01
  📁 Tamaño original: 0.49 MB
  📊 282 chunks, ~0.3 MB estimado
📄 Procesando 282 chunks con nueva estrategia PDF...
    ⚠️ No se encontró header disponible en los últimos 0 chunks
    📍 Encontrado sec en posición 1: "DEDICATION..."
    ✅ Chunk 2 (para): combinado con sec (1KB)
    📍 Encontrado sec en posición 3: "INTRODUCTORY NOTES..."
    ✅ Chunk 4 (para): combinado con sec (1KB)
    📍 Encontrado sec en posición 4: "INTRODUCTORY NOTES

It is with considera..."
    ✅ Chunk 5 (para): combinado con para, sec (1KB)
    📍 Encontrado sec en posición 5: "INTRODUCTORY NOTES

It is with considera..."
    ✅ Chunk 6 (para): combinado con para, sec (2KB)
    📍 Encontrado sec en posición 6: "INTRODUCTORY NOTES

It is with considera..."
    ✅ Chunk